In [13]:
%pip install --quiet -U "langchain-community>=0.2.16" langchain langchain-google-genai

In [14]:
import getpass
import os
import json
from typing import Dict, Any

In [15]:
if not os.environ.get("JINA_API_KEY"):
    os.environ["JINA_API_KEY"] = getpass.getpass("Enter your Jina API key: ")

if not os.environ.get("GOOGLE_API_KEY"):
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google/Gemini API key: ")

In [16]:
from langchain_community.tools import JinaSearch
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableConfig, chain
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage

print("🔧 Setting up tools and model...")

🔧 Setting up tools and model...


In [17]:
search_tool = JinaSearch()
print(f"✅ Jina Search tool initialized: {search_tool.name}")

print("\n🔍 Testing Jina Search directly:")
direct_search_result = search_tool.invoke({"query": "what is langgraph"})
print(f"Direct search result preview: {direct_search_result[:200]}...")

✅ Jina Search tool initialized: jina_search

🔍 Testing Jina Search directly:
Direct search result preview: [{"title": "LangGraph", "link": "https://www.langchain.com/langgraph", "snippet": "Build controllable agents with LangGraph, our low-level agent orchestration framework. Deploy and scale with LangGrap...


In [18]:
gemini_model = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0.1,
    convert_system_message_to_human=True
)
print("✅ Gemini model initialized")

✅ Gemini model initialized


In [19]:
detailed_prompt = ChatPromptTemplate.from_messages([
    ("system", """You are an intelligent assistant with access to web search capabilities.
    When users ask questions, you can use the Jina search tool to find current information.

    Instructions:
    1. If the question requires recent or specific information, use the search tool
    2. Provide comprehensive answers based on the search results
    3. Always cite your sources when using search results
    4. Be helpful and informative in your responses"""),
    ("human", "{user_input}"),
    ("placeholder", "{messages}"),
])

In [20]:
gemini_with_tools = gemini_model.bind_tools([search_tool])
print("✅ Tools bound to Gemini model")

main_chain = detailed_prompt | gemini_with_tools

def format_tool_result(tool_call: Dict[str, Any], tool_result: str) -> str:
    """Format tool results for better readability"""
    return f"Search Results for '{tool_call['args']['query']}':\n{tool_result[:800]}..."

✅ Tools bound to Gemini model


In [21]:
@chain
def enhanced_search_chain(user_input: str, config: RunnableConfig):
    """
    Enhanced chain that handles tool calls and provides detailed responses
    """
    print(f"\n🤖 Processing query: '{user_input}'")

    input_data = {"user_input": user_input}

    print("📤 Sending to Gemini...")
    ai_response = main_chain.invoke(input_data, config=config)

    if ai_response.tool_calls:
        print(f"🛠️  AI requested {len(ai_response.tool_calls)} tool call(s)")

        tool_messages = []
        for i, tool_call in enumerate(ai_response.tool_calls):
            print(f"   🔍 Executing search {i+1}: {tool_call['args']['query']}")

            tool_result = search_tool.invoke(tool_call)

            tool_msg = ToolMessage(
                content=tool_result,
                tool_call_id=tool_call['id']
            )
            tool_messages.append(tool_msg)

        print("📥 Getting final response with search results...")
        final_input = {
            **input_data,
            "messages": [ai_response] + tool_messages
        }
        final_response = main_chain.invoke(final_input, config=config)

        return final_response
    else:
        print("ℹ️  No tool calls needed")
        return ai_response

In [22]:
def test_search_chain():
    """Test the search chain with various queries"""

    test_queries = [
        "what is langgraph",
        "latest developments in AI for 2024",
        "how does langchain work with different LLMs"
    ]

    print("\n" + "="*60)
    print("🧪 TESTING ENHANCED SEARCH CHAIN")
    print("="*60)

    for i, query in enumerate(test_queries, 1):
        print(f"\n📝 Test {i}: {query}")
        print("-" * 50)

        try:
            response = enhanced_search_chain.invoke(query)
            print(f"✅ Response: {response.content[:300]}...")

            if hasattr(response, 'tool_calls') and response.tool_calls:
                print(f"🛠️  Used {len(response.tool_calls)} tool call(s)")

        except Exception as e:
            print(f"❌ Error: {str(e)}")

        print("-" * 50)

In [12]:
if __name__ == "__main__":
    print("\n🚀 Starting enhanced LangChain + Gemini + Jina Search demo...")
    test_search_chain()

    print("\n" + "="*60)
    print("💬 INTERACTIVE MODE - Ask me anything! (type 'quit' to exit)")
    print("="*60)

    while True:
        user_query = input("\n🗣️  Your question: ").strip()
        if user_query.lower() in ['quit', 'exit', 'bye']:
            print("👋 Goodbye!")
            break

        if user_query:
            try:
                response = enhanced_search_chain.invoke(user_query)
                print(f"\n🤖 Response:\n{response.content}")
            except Exception as e:
                print(f"❌ Error: {str(e)}")

🔧 Setting up tools and model...
✅ Jina Search tool initialized: jina_search

🔍 Testing Jina Search directly:
Direct search result preview: [{"title": "LangGraph", "link": "https://www.langchain.com/langgraph", "snippet": "Build controllable agents with LangGraph, our low-level agent orchestration framework. Deploy and scale with LangGrap...
✅ Gemini model initialized
✅ Tools bound to Gemini model

🚀 Starting enhanced LangChain + Gemini + Jina Search demo...

🧪 TESTING ENHANCED SEARCH CHAIN

📝 Test 1: what is langgraph
--------------------------------------------------

🤖 Processing query: 'what is langgraph'
📤 Sending to Gemini...


/usr/local/lib/python3.11/dist-packages/langchain_google_genai/chat_models.py:424: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


🛠️  AI requested 1 tool call(s)
   🔍 Executing search 1: langgraph
📥 Getting final response with search results...


/usr/local/lib/python3.11/dist-packages/langchain_google_genai/chat_models.py:424: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


✅ Response: LangGraph is a framework for building robust and stateful language agents. Here's a breakdown of what it is and its key aspects:

**1. Core Functionality:**

*   **Agent Orchestration:** LangGraph is designed to help build, manage, and deploy complex AI agent workflows. It provides a way to structur...
--------------------------------------------------

📝 Test 2: latest developments in AI for 2024
--------------------------------------------------

🤖 Processing query: 'latest developments in AI for 2024'
📤 Sending to Gemini...


...

🛠️  AI requested 1 tool call(s)
   🔍 Executing search 1: latest developments in AI for 2024
📥 Getting final response with search results...


/usr/local/lib/python3.11/dist-packages/langchain_google_genai/chat_models.py:424: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


✅ Response: Based on the search results, here are some of the latest developments in AI for 2024:

**1. AI Market and Investment:**

*   The AI boom drove record valuations and market growth in 2024. Companies like NVIDIA, Broadcom, and Microsoft saw strong revenue growth. NVIDIA's revenue from data center GPUs...
--------------------------------------------------

📝 Test 3: how does langchain work with different LLMs
--------------------------------------------------

🤖 Processing query: 'how does langchain work with different LLMs'
📤 Sending to Gemini...


/usr/local/lib/python3.11/dist-packages/langchain_google_genai/chat_models.py:424: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


ℹ️  No tool calls needed
✅ Response: LangChain is designed to be a versatile framework for working with various Language Models (LLMs). Here's how it achieves this:

1.  **LLM Abstraction:** LangChain provides a standard interface for interacting with different LLMs. This abstraction layer allows you to swap out one LLM for another wit...
--------------------------------------------------

💬 INTERACTIVE MODE - Ask me anything! (type 'quit' to exit)

🗣️  Your question: quit
👋 Goodbye!
